In [2]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import  RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import  mean_squared_error, r2_score

In [3]:
data = pd.read_csv("data/players_22.csv", dtype={25: str, 108: str})
data = data.drop(columns=[
    'player_url',
    'player_face_url',
    'club_logo_url',
    'club_flag_url',
    'nation_logo_url',
    'nation_flag_url'
])
Yvalue =   "overall"     # "value_eur" #"overall"
testSize = 0.2
randomnumber=21
data = data[data[Yvalue].notna()]
X = data.drop(Yvalue, axis=1)
y = data[Yvalue]


In [4]:
numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns
categorical_cols = X.select_dtypes(include=['object', 'category']).columns

In [5]:
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
])

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=testSize, random_state=randomnumber)

In [7]:
rfr = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestRegressor(random_state=randomnumber))
])

rfr.fit(X_train, y_train)
y_pred_rfr = rfr.predict(X_test)
mse = mean_squared_error(y_test, y_pred_rfr)
r2 = r2_score(y_test, y_pred_rfr)

print("Mean Squared Error (MSE) for Randon ForestRegressor:", mse)
print("R2 score for Randon ForestRegressor:", r2)

Mean Squared Error (MSE) for Randon ForestRegressor: 0.27750608108108116
R2 score for Randon ForestRegressor: 0.9941035482598629


In [13]:
y_pred_rfr = rfr.predict(X_train)
mse = mean_squared_error(y_train, y_pred_rfr)
r2 = r2_score(y_train, y_pred_rfr)
print("Mean Squared Error (MSE) for Randon ForestRegressor (Training dataset):", mse)
print("R2 score for Randon ForestRegressor (Training dataset):", r2)

Mean Squared Error (MSE) for Randon ForestRegressor (Training dataset): 0.03498185303099213
R2 score for Randon ForestRegressor (Training dataset): 0.9992620307190531


In [8]:
lr = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressioon', LinearRegression())
])

lr.fit(X_train, y_train)
y_pred_lr = lr.predict(X_test) 
mse = mean_squared_error(y_test, y_pred_lr)
r2 = r2_score(y_test, y_pred_lr)


print("Mean Squared Error (MSE) for Linear Regression:", mse)
print("R2 score for Linear Regression:", r2)

Mean Squared Error (MSE) for Linear Regression: 1.5832931403083335
R2 score for Linear Regression: 0.9663581729238198


In [14]:
y_pred_lr = lr.predict(X_train)
mse = mean_squared_error(y_train, y_pred_lr)
r2 = r2_score(y_train, y_pred_lr)


print("Mean Squared Error (MSE) for Linear Regression(Training dataset):", mse)
print("R2 score for Linear Regression(Training dataset):", r2)

Mean Squared Error (MSE) for Linear Regression: 8.746514921106902e-06
R2 score for Linear Regression: 0.9999998154854941


In [10]:
svr = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', SVR(kernel='rbf'))
])

svr.fit(X_train, y_train)
y_pred_svr = svr.predict(X_test)

mse = mean_squared_error(y_test, y_pred_svr)
r2 = r2_score(y_test, y_pred_svr)


print("Mean Squared Error (MSE) for SVR:", mse)
print("R2 score for SVR:", r2)


Mean Squared Error (MSE) for SVR: 1.1270164352543843
R2 score for SVR: 0.9760531445115354


In [16]:
y_pred_svr = svr.predict(X_train)

mse = mean_squared_error(y_train, y_pred_svr)
r2 = r2_score(y_train, y_pred_svr)


print("Mean Squared Error (MSE) for SVR(Training dataset):", mse)
print("R2 score for SVR(Training dataset):", r2)

Mean Squared Error (MSE) for SVR: 0.7627263247320212
R2 score for SVR: 0.9839096974959233


In [11]:


player_names = X_test['short_name']

results = pd.DataFrame({
    'Player': player_names,
    'True Value': y_test,
    'Predicted Value by RFR': y_pred_rfr,
    'Predicted Value by LR': y_pred_lr,
    'Predicted Value by SVR': y_pred_svr
})



In [12]:
results.to_excel('output.xlsx')